# Install Dependencies

In [ ]:
!pip install pmdarima
!pip install hydroeval

In [ ]:
import pandas as pd
import numpy as np
import hydroeval as he
import pickle
import os

from math import sqrt
from sklearn.metrics import mean_squared_error
from pandas import read_csv
from matplotlib import pyplot
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime

# Define Dataset Importer class

In [ ]:
class DatasetImporter:
  def __init__(self, csv_path, start_date, end_date):
    self.csv_path = csv_path
    self.start_date = start_date
    self.end_date = end_date
  
  # TODO: change default for endog_col_name
  def import_data(self, endog_col_name='krs_level_m', date_col_name='date'):
    self.dataset = read_csv(self.csv_path, header=0, dayfirst=True, parse_dates=[date_col_name], index_col=[date_col_name])
    # Construct arrays for endogenous / dependent variables
    self.df_endog = self.dataset.filter([endog_col_name])
    # Filter data from start_date to end_date
    self.df_endog = self.df_endog[self.start_date:self.end_date]
    # Get array from dataframe
    self.arr_endog = self.df_endog.values

    # Construct arrays for exogenous / independent variables
    # Drop the endog_col_name column
    self.df_exog = self.dataset.drop([endog_col_name], axis=1)
    # Filter data from start_date to end_date
    self.df_exog = self.df_exog[self.start_date:self.end_date]
    # Get array from dataframe
    self.arr_exog = self.df_exog.values

# Data Transformation methods

In [ ]:
# splits given data into train and test depending on the train_size
def train_test_split(data, train_size):
  train, test = data[:train_size], data[train_size:]
  return train, test

# converts given data into a step sequence of given step
def step_sequence_converter(data, step):
  step_seq = list()
  for i in range(len(data) - step):
    step_seq.append(data[i:i+step])
  return np.array(step_seq)

# split a univariate data into samples
def split_sequence(data, n_steps_in, n_steps_out):
	x, y = list(), list()
	for i in range(len(data)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(data):
			break
		# gather input and output parts of the pattern
		data_x, data_y = data[i:end_ix], data[end_ix:out_end_ix]
		x.append(data_x)
		y.append(data_y)
	return np.array(x), np.array(y)

# calculate test residuals by subtracting predicted values from actual values
def calculate_test_residuals(actuals, predictions):
  test_residuals = list()
  for i in range(len(predictions)):
    test_residuals.append(actuals[i] - predictions[i])
  return np.array(test_residuals)

# get column from array of tuples
def column_from_array_of_tuples(arr, column_no):
  return [elem[column_no] for elem in arr]

# Saving and Loading model methods

In [ ]:
# saves sarimax model using pickle
def save_pickle_model(model, filename):
  pickle.dump(model, open(f'{model_data_folder_path}/{filename}.sav', 'wb'))
  print(f'Model saved successfully => {model_data_folder_path}/{filename}.sav')

# loads sarimax model using pickle
def load_pickle_model(filepath):
  loaded_model = pickle.load(open(filepath, 'rb'))
  print(f'Model loaded successfully')
  return loaded_model

# saves MLP model
def save_NN_model(model, model_name):
  model.save(f'{model_data_folder_path}/{model_name}')
  print(f'MLP Model saved successfully => {model_data_folder_path}/{model_name}')

# Load saved MLP model
def load_NN_model(model_name):
  return keras.models.load_model(f'{model_data_folder_path}/{model_name}')

# Saving and Loading predictions methods

In [ ]:
# saves predictions array to csv
def save_predictions(predictions, filename):
  pd.DataFrame(predictions).to_csv(f'{model_data_folder_path}/{filename}.csv')
  print(f'Predictions saved successfully => {model_data_folder_path}/{filename}.csv')

# loads predictions from csv
def load_predictions(filepath):
  return read_csv(filepath,index_col=0)

# Error & Efficiency Calculation Methods

In [ ]:
# calculates root mean squared error or rmse
def measure_rmse(actual, predicted):
  return sqrt(mean_squared_error(actual, predicted))

# calculates nash sutcliffe efficiency or nse
def measure_nse(actual, predicted):
  nse = he.evaluator(he.nse, predicted, actual)
  return nse[0]

# Sarimax Model Methods

## Sarimax Training & Forecasting

In [ ]:
# one-step sarima forecast
def sarimax_forecast(model_fit, exog_forecast, steps):
  # make steps prediction and return the predictions
  prediction = model_fit.forecast(exog=exog_forecast, steps=steps)
  return prediction

# one-step sarima forecast
def sarimax_train_and_forecast(endog_train, exog_train, exog_forecast, order, seasonal_order, steps):
  # define model
  model = SARIMAX(endog_train, exog=exog_train, order=order, seasonal_order=seasonal_order)
  # fit model
  model_fit = model.fit()
  # make one step forecast which would give n_steps forecasts
  prediction = model_fit.forecast(exog=exog_forecast, steps=steps)
  return prediction, model_fit

## Sarimax Weekly Walk Forward Validation

In [ ]:
# walk-forward validation for arima
def sarimax_weekly_walk_forward_validation(endog, exog, order, seasonal_order, steps, train_size):
  model_fit=None
  predictions = list()
  # split dataset
  endog_train, endog_test = train_test_split(endog, train_size)
  exog_train, exog_test = train_test_split(exog, train_size)
  # seed history with training dataset
  history_endog = [x for x in endog_train]
  history_exog = [x for x in exog_train]
  # step over each time-step in the test set
  for i in range(len(endog_test) - steps):
      if i % 7 == 0:
        # retrain model on history_endog and forecast for next steps
        # fit model and make forecast for history
        (prediction, model_fit) = sarimax_train_and_forecast(
            endog_train=history_endog,
            exog_train=history_exog,
            exog_forecast=exog_test[i:i + steps],
            order=order,
            seasonal_order=seasonal_order,
            steps=steps,
            )
        # print(f"RetrainAndPredictionRun {i}/{len(endog_test)} ===> prediction = {prediction}")
        # store forecast in list of predictions
        predictions.append(prediction)
        # add actual observation to history for the next loop
        history_endog.append(endog_test[i])
        history_exog.append(exog_test[i])
      else:
        # do not retrain model. Just make n_steps_out + (i%7) forecasts
        prediction = sarimax_forecast(
            model_fit=model_fit, 
            exog_forecast=exog_test[int(i/7)*7:i + steps],
            steps=(steps + (i%7))
            )
        # store last (steps) forecasts to the predictions array
        predictions.append(prediction[-steps:])
        # add actual observation to history for the next loop
        history_endog.append(endog_test[i])
        history_exog.append(exog_test[i])
      print(f'Run {i}/{len(endog_test) - steps} complete')
  # estimate prediction error
  rmse = measure_rmse(step_sequence_converter(endog_test.reshape(len(endog_test,)), steps), predictions)
  nse = measure_nse(step_sequence_converter(endog_test.reshape(len(endog_test,)), steps).flatten(), np.array(predictions).flatten())
  return model_fit, predictions, rmse, nse

# MLP Model Methods

## MLP Training and Forecasting

In [ ]:
# one-step MLP forecast
def mlp_train_and_forecast(train_x, train_y, test_x, n_steps_in, n_steps_out, dense_units, epochs):
  # define model
  model = Sequential()
  model.add(Dense(dense_units, activation='relu', input_dim=n_steps_in))
  model.add(Dense(n_steps_out))
  model.compile(optimizer='adam', loss='mse')
  # fit model
  model.fit(train_x, train_y, epochs=epochs, verbose=0)
  prediction = model.predict(test_x, verbose=0)
  return prediction, model

def mlp_forecast(model, test_x):
  prediction = model.predict(test_x, verbose=0)
  return prediction

## MLP Weekly Walk Forward Validation

In [ ]:
# walk-forward validation for MLP
def mlp_weekly_walk_forward_validation(train, test, n_steps_in, n_steps_out, dense_units=100, epochs=100):
  model_fit=None
  predictions = list()
  # split data into samples
  train_x, train_y = split_sequence(train, n_steps_in, n_steps_out)
  test_x, test_y = split_sequence(test, n_steps_in, n_steps_out)
  # seed history with training dataset
  history = train_x
  # step over each time-step in the test set
  for i in range(len(test_x)):
      print(f"Run {i+1}/{len(test_x)}")
      if i%7 == 0:
        # fit model and make forecast for history
        (prediction, model_fit) = mlp_train_and_forecast(
            train_x=train_x,
            train_y=train_y,
            test_x=test_x[i].reshape((1, n_steps_in)),
            n_steps_in=n_steps_in,
            n_steps_out=n_steps_out,
            dense_units=dense_units,
            epochs=epochs
            )
        # store forecast in list of predictions
        predictions.append(prediction[0])
      else:
        # do not retrain model, just make forecast
        prediction = mlp_forecast(
            model=model_fit,
            test_x=test_x[i].reshape((1, n_steps_in))
        )
        # store forecast in list of predictions
        predictions.append(prediction[0])
      # add actual observation to history for the next loop
      np.append(history, test_y[i])
  return model_fit, predictions

# Repeated Evaluation Methods

In [ ]:
# repeat evaluation of a config
# eg : repeat_evaluate(lambda: run_sarimax(arguments), n_repeats=30)
def repeat_evaluate(model_runner, n_repeats=3):
  # fit and evaluate the model n_repeats times
  results = [model_runner() for _ in range(n_repeats)]
  # results is an array of tuple (trained model, predictions, rmse) 
  # get array of train_models from result
  arr_trained_models = column_from_array_of_tuples(results, 0)
  # get array of predictions from result
  arr_predictions = column_from_array_of_tuples(results, 1)
  # get array of rmse from result
  arr_rmse = column_from_array_of_tuples(results, 2)
  return (arr_trained_models, arr_predictions, arr_rmse)


# summarize model performance
def summarize_scores(name, scores):
	# print a summary
	scores_m, score_std = np.mean(scores), np.std(scores)
	print('%s: %.3f RMSE (+/- %.3f)' % (name, scores_m, score_std))
	# box and whisker plot
	pyplot.boxplot(scores)
	pyplot.show()

# Runner - KRS

## Define Constants

In [ ]:
# Note: Both start_date and end_date get included
start_date = '2014-01-01'
end_date = '2021-03-01'
n_steps_in = 1
n_steps_out = 30
train_size = 1826

# define sarimax order and seasonal order
sarimax_order = (1, 1, 0) # p,d,q
sarimax_seasonal_order = (1, 0, 1, 7) # P,D,Q,m

In [ ]:
# Constants for saving model runs
model_data_folder_path = './output'
os.makedirs(model_data_folder_path)

## Data imports

In [ ]:
# import data
krsCsvPath = './krs_data.csv'
krsImporter = DatasetImporter(krsCsvPath, start_date, end_date)
krsImporter.import_data()
# get endog and exog arrays
endog = krsImporter.arr_endog
exog = krsImporter.arr_exog
print(f'Shape of endogenous array = {endog.shape}')
print(f'Shape of exogenours array = {exog.shape}')

## Add fourier terms to exogenous array

In [ ]:
krs_df_exog = krsImporter.df_exog

# construct fourier columns
fourier_exog = pd.DataFrame({'date': krsImporter.df_endog.index})
fourier_exog = fourier_exog.set_index(pd.PeriodIndex(fourier_exog['date'], freq='D'))
fourier_exog['sin365'] = np.sin(2 * np.pi * fourier_exog.index.dayofyear / 365.25)
fourier_exog['cos365'] = np.cos(2 * np.pi * fourier_exog.index.dayofyear / 365.25)
fourier_exog = fourier_exog.drop(columns=['date'])

# set fourier columns for river level exogenous variables
krs_df_exog['sin365'] = fourier_exog['sin365'].values
krs_df_exog['cos365'] = fourier_exog['cos365'].values

exog = krs_df_exog.values
print(f'Shape of exogenours array = {exog.shape}')

## Run Sarimax Weekly Walk Forward Validation

### Run SARIMAX, Calculate and print predictions, error and efficiency

In [ ]:
(sarimax_model, 
sarimax_predictions, 
sarimax_rmse, 
sarimax_nse) = sarimax_weekly_walk_forward_validation(
    endog, 
    exog, 
    sarimax_order, 
    sarimax_seasonal_order, 
    n_steps_out, 
    train_size)
print(f"Predictions shape = {pd.DataFrame(sarimax_predictions).shape}")
print(f"NSE = {sarimax_nse}")
print(f"RMSE = {sarimax_rmse}")

### Save SARIMAX trained model and predictions

In [ ]:
save_predictions(sarimax_predictions, 'sarimax_predictions')
save_pickle_model(sarimax_model, 'sarimax_model')

## Run MLP

### Run MLP model

In [ ]:
# calculate train residuals
train_residuals = sarimax_model.resid[:train_size-1]

# calculate test residuals
train, test = train_test_split(endog, train_size=train_size)
actuals = test[0: len(test) - n_steps_out].reshape(len(test) - n_steps_out,)
first_predictions = np.array(sarimax_predictions)[:,0]
test_residuals = calculate_test_residuals(actuals, first_predictions)

# run mlp weekly walk forward validation
(mlp_model, mlp_predictions) = mlp_weekly_walk_forward_validation(
    train=train_residuals, 
    test=test_residuals, 
    n_steps_in=n_steps_in, 
    n_steps_out=n_steps_out
    )

### Save MLP Predictions and trained MLP Model

In [ ]:
save_predictions(mlp_predictions, 'mlp_predictions')
save_NN_model(mlp_model, 'mlp_model')

## Compute final predictions by adding the linear and non linear components

***final_prediction = linear (arima/sarimax prediction) + non-linear (NN prediction)***


###  Compute final predictions

In [ ]:
final_predictions = np.add(sarimax_predictions[:len(mlp_predictions)], mlp_predictions)
print(f'Shape of final predictions = {np.shape(final_predictions)}')

### Store final predictions

In [ ]:
save_predictions(final_predictions, 'hybrid_model_predictions')

## Compute RMSE & NSE of Hybrid Model

In [ ]:
# compute actual values
actual = step_sequence_converter(test.reshape(len(test),), n_steps_out)[:len(final_predictions)]
# compute and print RMSE
final_rmse = measure_rmse(actual, final_predictions)
print(f"RMSE of hybrid model = {final_rmse}")
# compute and print NSE
final_nse = measure_nse(np.array(actual).flatten(), np.array(final_predictions).flatten())
print(f"NSE of hybrid model = {final_nse}")